# Evaluation: Precision & Recall
## Using the evaluation metrics we have learned, we are going to compare how well some different types of classifiers perform on different evaluation metrics
### We are going to use a dataset of written numbers which we can import from sklearn. Run the code below to do so. 


In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

mnist = fetch_openml('mnist_784')
X, y = mnist['data'], mnist['target']

### Now take a look at the shapes of the X and y matricies 

In [2]:
X.data.shape

(70000, 784)

In [3]:
y.data.shape

(70000,)

In [4]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
y

array(['5', '0', '4', ..., '4', '5', '6'], dtype=object)

In [6]:
X.dtype

dtype('float64')

### Now, let's pick one entry and see what number is written. Use indexing to pick the 36000th digit

In [7]:
X[35999:36001]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [8]:
y[35999]

'0'

### You can use the .reshape(28,28) function and plt.imshow() function with the parameters cmap = matplotlib.cm.binary, interpolation="nearest" to make a plot of the number. Be sure to import matplotlib!

In [10]:
#interpolation = 'nearest' works well when a small image is scaled up
#interpolation = 'none' works well when a big image is scaled down


fig, ax_array = plt.subplots(28, 28)
axes = ax_array.flatten()

ax.imshow(X.data[i],interpolation='nearest', cmap='gray_r')

plt.tight_layout(h_pad=0.5, w_pad=0.01)
plt.show()

NotImplementedError: multi-dimensional sub-views are not implemented

Error in callback <function flush_figures at 0x7f6a0004c8c0> (for post_execute):


KeyboardInterrupt: 

In [ ]:
h = int(np.sqrt(samples))
w = h

f, axr = plt.subplots(h, w, figsize=figsize)
    
    for i in range(h):
        for j in range(w):
            index = np.random.choice(len(images))
            axar[i, j].imshow(images[index])
            axar[i, j].axis('off')
            
    plt.tight_layout()
    plt.show()

In [11]:
def images_plot (images: np.array,
                labels: np.array,
                samples: int,
                figsize: tuple = (8,8)):
    
    
    

In [ ]:
images_plot(train_images, 
                   [class_names[i] for i in train_labels], 
                   samples=25)

### Use indexing to see if what the plot shows matches with the outcome of the 36000th index

### Now lets break into a test train split to run a classification. Instead of using sklearn, use indexing to select the first 60000 entries for the training, and the rest for training.

### We are going to make a two-class classifier, so lets restrict to just one number, for example 5s. Do this by defining a new y training and y testing sets for just the number 5

### Lets train a logistic regression to predict if a number is a 5 or not (remember to use the 'just 5s' y training set!)

### Does the classifier predict correctly the 36000th digit we picked before?

### To make some comparisons, we are going to make a very dumb classifier, that never predicts 5s. Build the classifier with the code below, and call it using: never_5_clf = Never5Classifier()

In [13]:
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
    def fit(self, X, y=None):
        pass
    def predict(self, X):
        return np.zeros((len(X), 1), dtype=bool)

never_5_clf = Never5Classifier()

### Now lets fit and predict on the testing set using our never 5 Classifier

### Let's compare this to the Logistic Regression. Examine the confusion matrix, precision, recall, and f1_scores for each. What is the probability cutoff you are using to decide the classes?

### What are the differences you see? Without knowing what each model is, what can these metrics tell you about how well each works?

### Now let's examine the roc_curve for each. Use the roc_curve method from sklearn.metrics to help plot the curve for each

### Now find the roc_auc_score for each. 

### What does this metric tell you? Which classifier works better with this metric in mind?